In [ ]:
# 🐣 Please follow me for new updates https://twitter.com/camenduru
# 🔥 Please join our discord server https://discord.gg/k5BwmmvJJU
# 🥳 Please join my patreon community https://patreon.com/camenduru

%cd /kaggle/working
!git clone -b dev https://github.com/camenduru/CCSR
%cd /kaggle/working/CCSR

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/CCSR/resolve/main/real-world_ccsr.ckpt -d /kaggle/working/CCSR/weights -o real-world_ccsr.ckpt

!pip install  https://download.pytorch.org/whl/cu118/xformers-0.0.23.post1%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl
!pip install -q einops pytorch-lightning omegaconf open-clip-torch gradio gradio_imageslider pyngrok
!pip install -q -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip uninstall -y torchaudio

!rm -rf /kaggle/working/CCSR/preset/test_datasets/*

%cd /kaggle/working/CCSR
import gradio as gr
import subprocess
import shutil, os
from gradio_imageslider import ImageSlider

def generate(image, seed):
    print(image, seed)
    os.system(f"rm /kaggle/working/CCSR/experiments/test/sample0/input.png")
    os.system(f"rm -rf /kaggle/working/CCSR/preset/test_datasets")
    os.system(f"mkdir -p /kaggle/working/CCSR/preset/test_datasets")
    shutil.copy(image, '/kaggle/working/CCSR/preset/test_datasets/input.png')
    command = f"python inference_ccsr.py --input preset/test_datasets --config configs/model/ccsr_stage2.yaml --ckpt weights/real-world_ccsr.ckpt --steps 45 --sr_scale 4 --t_max 0.6667 --t_min 0.3333 --color_fix_type adain --output experiments/test --device cuda --repeat_times 1"
    try:
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("Output:", result.stdout)
        output_path = '/kaggle/working/CCSR/experiments/test/sample0/input.png'
        return (image, output_path)
    except subprocess.CalledProcessError as e:
        print("Error:", e.stderr)
        return None

with gr.Blocks() as demo:
    with gr.Row():
      with gr.Column():
          image = gr.Image(type='filepath')
          seed = gr.Slider(minimum=0, maximum=1000000, value=643534)
          button = gr.Button()
#       output_image = gr.Image(show_label=False, type="filepath", interactive=False, height=512, width=512)
      output_image = ImageSlider(show_label=False, type="filepath", interactive=False, height=512, width=512)
    button.click(fn=generate, inputs=[image, seed], outputs=[output_image])

from pyngrok import ngrok, conf
conf.get_default().auth_token = 'NGROK_TOKEN_HERE'
public_url = ngrok.connect(7861).public_url
print(public_url)

demo.queue().launch(server_port=7861, inline=False, share=False, debug=True)